# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from markupsafe import Markup 

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mar del plata,-38.0023,-57.5575,59.02,80,0,20.71,AR,1697407203
1,1,puerto baquerizo moreno,-0.9000,-89.6000,81.59,69,40,10.36,EC,1697407088
2,2,albany,42.6001,-73.9662,53.06,77,100,3.00,US,1697407047
3,3,uvinza,-5.1036,30.3911,71.46,82,94,6.22,TZ,1697407284
4,4,bethel,41.3712,-73.4140,56.75,70,97,8.08,US,1697407284


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
            "Lng",
            "Lat",
            geo=True, 
            tiles="OSM",
            size = "Humidity",
            color = "City")

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Probably use cities with low humidity and mild temp in FARENHEIT
# remember to run the frame above that reads the CSV into the city_data_df dataframe
# before this cell can run properly

humidity_criteria = 60
temperature_criteria = 60

#Get a new df with filtered 
filtered_cities_df = city_data_df[(city_data_df["Humidity"] < humidity_criteria) & (city_data_df["Max Temp"] > temperature_criteria)]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df.tail()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
557,557,almeirim,-1.5233,-52.5817,85.10,54,60,3.06,BR,1697407418
563,563,yumbel,-37.0833,-72.5667,62.83,54,48,10.13,CL,1697407419
573,573,ardakan,32.3100,54.0175,66.49,32,8,4.97,IR,1697407422
574,574,taoudenni,22.6783,-3.9836,86.61,16,0,8.03,ML,1697407422
579,579,camiri,-20.0500,-63.5167,96.58,26,10,6.91,BO,1697407423


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Use the city_data_df dataframe above to copy into hotel_df just to be safe
hotel_df = city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,mar del plata,AR,-38.0023,-57.5575,80,
1,puerto baquerizo moreno,EC,-0.9000,-89.6000,69,
2,albany,US,42.6001,-73.9662,77,
3,uvinza,TZ,-5.1036,30.3911,82,
4,bethel,US,41.3712,-73.4140,70,
...,...,...,...,...,...,...
579,camiri,BO,-20.0500,-63.5167,26,
580,minas de marcona,PE,-15.2119,-75.1103,70,
581,cockburn town,TC,21.4612,-71.1419,66,
582,saldanha,ZA,-33.0117,17.9442,81,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
#TESTING ONLY - DO NOT ACTIVATE THIS CELL
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}
base_url = "https://api.geoapify.com/v2/places"

lat = "-0.9000"
lng = "-89.6000"

# Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
params["filter"] = f"circle:{lng},{lat},{radius}" 
params["bias"] = f"proximity:{lng},{lat}"

name_address = requests.get(base_url, params=params).json()
print(name_address)



{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': "Dory's House", 'country': 'Ecuador', 'country_code': 'ec', 'state': 'Galápagos', 'county': 'Cantón San Cristóbal', 'city': 'Parroquia Puerto Baquerizo Moreno', 'street': 'Roberto Koch', 'lon': -89.6061302, 'lat': -0.9007611, 'state_code': 'W', 'formatted': "Dory's House, Roberto Koch, Parroquia Puerto Baquerizo Moreno, Ecuador", 'address_line1': "Dory's House", 'address_line2': 'Roberto Koch, Parroquia Puerto Baquerizo Moreno, Ecuador', 'categories': ['accommodation', 'accommodation.hotel'], 'details': [], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': "Dory's House", 'osm_id': 3077654083, 'tourism': 'hotel', 'osm_type': 'n'}}, 'distance': 687, 'place_id': '518c8752d6ca6656c0595e2034f108d3ecbff00103f901434671b70000000092030c446f7279277320486f757365'}, 'ge

In [10]:
# Set parameters to search for a hotel
radius = 10000 # Should I still use meters even though we use Imperial units? 
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
radius = 10000 # Should I still use meters even though we use Imperial units? 
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search...")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longtitude = row["Lng"]
    latitude = row ["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longtitude},{latitude},{radius}" # it's 10000 meter radius!
    params["bias"] = f"proximity:{longtitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search...
mar del plata - nearest hotel: Nuevo Ostende
puerto baquerizo moreno - nearest hotel: Dory's House
albany - nearest hotel: No hotel found
uvinza - nearest hotel: Mapito lodge
bethel - nearest hotel: Hampton Inn Danbury
nantucket - nearest hotel: No hotel found
bubaque - nearest hotel: Chez Julio
sooke - nearest hotel: Best Western Premier Prestige Oceanfront Resort
ulladulla - nearest hotel: Hotel Marlin
adamstown - nearest hotel: No hotel found
chimbote - nearest hotel: Hostal Diamante Chimbote
pasuquin - nearest hotel: No hotel found
venice - nearest hotel: El Patio
waitangi - nearest hotel: Hotel Chathams
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
jamestown - nearest hotel: DoubleTree Jamestown
edinburgh of the seven seas - nearest hotel: No hotel found
weno - nearest hotel: High Tide Hotel
kirakira - nearest hotel: No hotel found
bredasdorp - nearest hotel: Victoria Hotel
ancud - nearest hotel: Cabañas
blackmans bay - nearest hote

,City,Country,Lat,Lng,Humidity,Hotel Name
0,mar del plata,AR,-38.0023,-57.5575,80,Nuevo Ostende
1,puerto baquerizo moreno,EC,-0.9000,-89.6000,69,Dory's House
2,albany,US,42.6001,-73.9662,77,No hotel found
3,uvinza,TZ,-5.1036,30.3911,82,Mapito lodge
4,bethel,US,41.3712,-73.4140,70,Hampton Inn Danbury
...,...,...,...,...,...,...
579,camiri,BO,-20.0500,-63.5167,26,Sol Chaqueño
580,minas de marcona,PE,-15.2119,-75.1103,70,No hotel found
581,cockburn town,TC,21.4612,-71.1419,66,The Salt Raker Inn
582,saldanha,ZA,-33.0117,17.9442,81,Hoedjesbaai


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display
# # Display sample data
# hotel_df

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", "Lat", geo=True, tiles="OSM",
    size="Humidity", color="City", hover_cols=["Hotel Name","Country"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)